In [9]:
import openai
import nbformat
import json
import os
import re
import datetime

client = openai.OpenAI()

def generate_notebook_code(prompt):
    """Generates a Jupyter Notebook in JSON format using the OpenAI API."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that generates Jupyter Notebooks in JSON format.  Return ONLY the JSON, with no surrounding text or code fences.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.2,
            max_tokens=2000,
        )
        response_content = response.choices[0].message.content

        # More robust JSON extraction: try direct parsing first, then regex
        try:
            return json.loads(response_content)  # Try direct parsing
        except json.JSONDecodeError:
            # If direct parsing fails, try regex as a fallback
            match = re.search(r"```json\n(.*)```", response_content, re.DOTALL)
            if match:
                json_string = match.group(1).strip()
                try:
                    return json.loads(json_string)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON (regex): {e}")
                    print(f"Raw JSON string: {json_string}")
                    return None
            else:
                print("No JSON object found in the response.")
                print(f"Response content: {response_content}") # Print full response
                return None

    except openai.APIError as e:
        print(f"An OpenAI API error occurred: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

def create_notebook(notebook_json, filename="generated_notebook.ipynb"):
    """Creates a .ipynb file from a JSON object."""
    if notebook_json:
        try:
            nb = nbformat.reads(json.dumps(notebook_json), as_version=4)
            with open(filename, "w") as f:
                nbformat.write(nb, f)
            print(f"Notebook '{filename}' created successfully.")
        except Exception as e:
            print(f"Error creating notebook: {e}")
    else:
        print("No notebook content to write.")

def get_valid_filename(prompt):
    """Extracts a suggested filename from the prompt."""
    match = re.search(r"create a notebook named `([^`]+)`", prompt, re.IGNORECASE)
    if match:
        return match.group(1).strip() + ".ipynb"
    return "generated_notebook.ipynb"  # Default filename


# --- Example Prompt (Improved) ---
prompt = """
Please create a notebook named `data_analysis` that performs the following:

1. **Imports**:
   - pandas
   - numpy
   - matplotlib.pyplot
   - seaborn

2. **Functions**:
   - `create_synthetic_data(n_samples=100)`:  Generates a pandas DataFrame with 'date' (daily from 2023-01-01) and 'value' (random normal) columns.
   - `plot_data(df)`:  Creates a line plot of the 'value' column against 'date'.

3. **Data Preparation**:
   - Call `create_synthetic_data` with `n_samples=150` to create a DataFrame named `df`.

4. **Analysis & Visualization**:
   - Display the first 5 rows of `df`.
   - Call `plot_data(df)`.
   - Create a correlation heatmap of `df`.

5.  Output a complete, valid Jupyter Notebook in JSON format.  Do not include any surrounding text. The notebook should run without errors.
"""

# --- Main Execution ---
# Get current datetime
now = datetime.datetime.now()

# Format the datetime as Year-Month-Day_Hour-Minute-Second
filename = now.strftime("autogenerated_notebook_%Y%m%d_%H%M%S.ipynb")
notebook_code = generate_notebook_code(prompt)
create_notebook(notebook_code, filename)

Notebook 'autogenerated_notebook_20250219_195914.ipynb' created successfully.
